# Brachyury Segmentation

Loading of brachyury images and brightfield masks

In [1]:
from skimage import io
from skimage.filters import threshold_otsu
import os
import numpy as np
import organoid_prediction_python as opp

home_directory = r"C:\Users\savill\OneDrive\Documents\PhD Jesse\Embryonic_organoid_prediction\Processing dataset for Paper\TLS_2D_morphostate_investigation/"

brightfield_mask_location = home_directory +  r"image_data\Processed Data\Masks Brightfield\96h"
brachyury_image_location = home_directory + r"image_data\Processed Data\Focussed Images\96h\Brachyury"

bra_image_collection = io.imread_collection(brachyury_image_location+"/*.tif")
bra_file_names = [os.path.split(fname)[1] for fname in bra_image_collection.files]

bf_image_collection = io.imread_collection(brightfield_mask_location+"/*.tif")
bf_file_names = [os.path.split(fname)[1] for fname in bf_image_collection.files]

Small sanity check if both stacks have the same order

In [2]:
[name[:-8] for name in bra_file_names] == [name[:-12] for name in bf_file_names]

True

Segmentation of brachyury as implemented in opp

In [3]:
%%time

thresh = threshold_otsu(np.array(bra_image_collection))
print(f"global_threshold: {thresh}")
segmented_images = opp.segment_brachyury_stack(bf_masks=np.array(bf_image_collection),bra_images=np.array(bra_image_collection),thresh = thresh)

global_threshold: 16.66704750061035
CPU times: total: 8.83 s
Wall time: 14.7 s


In [4]:
segmented_images.shape

(576, 598, 712)

In [5]:
out_path = home_directory + "image_data/Processed Data/Masks Brachyury/96h"

if not os.path.exists(out_path):
   os.makedirs(out_path)
   print("The new directory is created!")
    
for name, mask in zip(bra_file_names,segmented_images):
    name_mask = f"{name[:-4]}_mask.tif"
    io.imsave(f"{out_path}/{name_mask}",mask.astype("uint8"))

Another sanity check

In [6]:
import napari
viewer = napari.Viewer()
viewer.add_image(np.array(bra_image_collection))
viewer.add_labels(segmented_images)

<Labels layer 'segmented_images' at 0x17614a74b50>